In [7]:
from scipy.stats import mode
import time
import statistics
import operator
import cv2
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
def get_data(read_path,filter_count=0,filter_type='m'):
    
    #Load the image in black and white (0 - b/w, 1 - color).
    img = cv2.imread(read_path, 0)

    #Get the height and width of the image.
    h, w = img.shape[:2]

    
    #Invert the image to be white on black for compatibility with findContours function.
    imgray = 255 - img
    #ret, imgray = cv2.threshold(imgray, 80, 255, cv2.THRESH_BINARY_INV)
    size=9+2*filter_count
    if (filter_type=='m'):
        thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,size,2)
    else:
        thresh = cv2.adaptiveThreshold(imgray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,size,2)

    #Find all the contours in thresh. In your case the 3 and the additional strike
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #Calculate bounding rectangles for each contour.
    rects = [cv2.boundingRect(cnt) for cnt in contours]
    #print(len(rects))
    
    rects2=[]
    for i in rects:
        if (i[2]>60 and i[3]>60):
            rects2.append(i)
        
    #Calculate the combined bounding rectangle points.
    top_x = min([x for (x, y, w, h) in rects2])
    top_y = min([y for (x, y, w, h) in rects2])
    bottom_x = max([x+w for (x, y, w, h) in rects2])
    bottom_y = max([y+h for (x, y, w, h) in rects2])
#     print(top_x)
#     print(top_y)
#     print(bottom_x)
#     print(bottom_y)
    
    #h_mode = statistics.mode([h for (x, y, w, h) in rects2])
    h_mode = mode([h for (x, y, w, h) in rects2])[0]
    
    #print(h_mode)
    
    n=int(((bottom_y-top_y)/int(h_mode)))
    #gap=(bottom_y-top_y)-(h_mode*20)
    #n=int(((bottom_y-top_y)-(gap))/h_mode)
    #print('Mode',h_mode)
    #print(n)
    
    temp=[]
    b = [[] for i in range(0, n+1)]
    for j in range (1,n+1):
        for i in rects2:
            x=i[0]
            y=i[1]
            w=i[2]
            h=i[3]
            if ((y > bottom_y-h_mode*j-h_mode/2) and (i not in temp)):
                b[j].append(i)
                temp.append(i)
            b[j]=sorted(b[j],key=lambda x:(-x[0]))
    return(imgray,b)

In [9]:
df=pd.read_excel(r"C:\Users\Rahul\Desktop\Capstone\OCR\scan copy\Labels.xlsx",index_col="Sr. No.")

In [13]:
read_path=r"C:\Users\Rahul\Desktop\Capstone\OCR\scan copy\scan_19_1"+"\\"
class_path=r'C:\Users\Rahul\Desktop\Capstone\OCR\data\results11'+'\\'
imgsave=1
for folder in range(1,621):
    class_folder=class_path+str(df.loc[folder,"Labels2"])
    if not os.path.exists(class_folder):
        os.makedirs(class_folder) 
m=g=0
c=[]

for i in range(2,30):
    if (i==18 or i==21):
        continue
    if (m==10 and g==10 and len(c)!=620):
        print("Skipped image is ", (i-1))
    m=g=0
    c=[]
    while((len(c)!=620) and (m!=10 or g!=10)):
        c=[]
        if (m<10 and g==0):
            m+=1
            imgray,b=get_data(read_path+str(i)+".jpg",filter_count=m,filter_type='m')
            #print(m)
        elif (m==10 and g<10):
            g+=1
            imgray,b=get_data(read_path+str(i)+".jpg",filter_count=g,filter_type='g')
            #print(g)  
        for row in b:
            for box in row:
                try:
                    w=box[2]
                    h=box[3]
                    #print(w)
                    if ((w>60 and w<80) and (h>60 and h <80)):
                        c.append(box)
                        #print(c)
                except:
                    continue
                    
        print(len(c))
        if (len(c)==620):
            print("here")
            count=1
            for box in c:
                x=box[0]
                y=box[1]
                w=box[2]
                h=box[3]
                class_folder=class_path+str(df.loc[count,"Labels2"])
                cv2.imwrite(class_folder+'\\'+str(i)+" img "+str(imgsave)+".jpg",imgray[y:y+h,x:x+w])
                imgsave+=1
                time.sleep(0.005)
                count+=1

39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
39
Skipped image is  2
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
42
Skipped image is  3
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
607
Skipped image is  4
612
612


KeyboardInterrupt: 